<hr style="border-width:4px; border-color:coral"></hr>

# Practice

<hr style="border-width:4px; border-color:coral"></hr>

This notebook will cover using a Pool of workers. 




### Review :  Monte Carlo simulations

<hr style="border-width:4px; border-color:coral"></hr>

Imagine a $2 \times 2$ foot dart board in with a circular target of radius 1 foot.  You throw darts at the dart board, each time counting how many times your darts hit the target.  If you throw enough darts at the dart board, you can approximate $\pi$. 

* How can you use this dart board game to approximate $\pi$?  

* Write a code that implements this idea in serial

* How can you use the multiprocessing moddule to speed up your approximation? 

### To Do

* Represent darts as randomly generated points $(x_i,y_i)$ in the domain $[-1,1]\times [-1,1]$.  Compute the distance of the randomly generated point to the origin to determine if the point is in the circle or not. 

* Speed up your calculation by using the multiprocessing module.  

### Question

* How close to pi can you get? 

In [1]:
import multiprocessing as mp
from numpy import *
from random import seed
from numpy.random import rand
mp.set_start_method('fork')


In [2]:
def darts(M,s,nval):
    # Generate random points in [-1,1]x[-1,1]
    seed(s)
    x = -1 + 2*rand(M)
    y = -1 + 2*rand(M)
    r = sqrt(x**2 + y**2)
    #print(x[0],y[0])

    in_target = less_equal(r,1)
    nz = count_nonzero(in_target)
    #print(f"Number of darts that hit the target : {nz} out of {M}")
    
    nval.value+=nz 
    #return nz

def compute_pi(M0,P):
    jobs=[]
    seedlist=[random.randint(1,1234567) for i in range(P)]
    
    nval=mp.Value('i',0) #shared array
    
    for i in range(P):
        p=mp.Process(target=darts,args=(M0,seedlist[i],nval))
        jobs.append(p)
        
    for p in jobs:
        p.start()
        
    for p in jobs:
        p.join()
    
    pi_approx=4*nval.value/(P*M0)
    
    return pi_approx

# Generate random points (x,y)
M = 2**20
#I=darts(M)
P=2
pi_approx=compute_pi(M,P)

#Approximate pi ...
#pi_approx=4*I/M
print(f"PI is approx {pi_approx}")
err=abs(pi-pi_approx)
print(f"Error is {err:.4e}")

PI is approx 3.1384353637695312
Error is 3.1573e-03


### Pool of workers

<hr style="border-width:4px; border-color:coral"></hr>


### Example : Pool.map

<hr style="border-width:4px; border-color:black"></hr>

Notice how results are returned from the process. 

### To do :

* Modify the code below to compute pi. 

In [3]:
import time 
def naptime(t):
    pname = mp.current_process().name
    print(f"{pname} is sleeping for {t:.4f} seconds\n")
    time.sleep(t)
    
    # Results can be returned from the process. 
    return t


nprocs = 4
pool = mp.Pool(processes=nprocs)

# Blocks until all jobs are finished
sleep_times = 2*rand(10)
results = pool.map(func=naptime,iterable=sleep_times)

print(results)
print("All done!")


ForkPoolWorker-3 is sleeping for 0.4108 seconds
ForkPoolWorker-4 is sleeping for 0.6005 seconds
ForkPoolWorker-6 is sleeping for 1.5372 seconds
ForkPoolWorker-5 is sleeping for 1.7486 seconds




ForkPoolWorker-3 is sleeping for 0.7506 seconds

ForkPoolWorker-4 is sleeping for 0.1665 seconds

ForkPoolWorker-4 is sleeping for 0.2519 seconds

ForkPoolWorker-4 is sleeping for 0.6801 seconds

ForkPoolWorker-3 is sleeping for 1.1047 seconds

ForkPoolWorker-6 is sleeping for 1.9283 seconds

[0.41079629421842334, 0.6005095594926331, 1.7485597220028537, 1.5372135479761966, 0.75064435533606, 0.1664808998106042, 0.25189601001416695, 0.6801163785084003, 1.1047278000342633, 1.9283256306425904]
All done!


### Example : Pool.async_map

<hr style="border-width:4px; border-color:black"></hr>

In [4]:
async_results = pool.map_async(func=f, iterable=data, 
    callback=cb)

# Blocks until results are ready
results = async_results.get()    

NameError: name 'f' is not defined